In [1]:
# Prepare Chat Completion Dataset for SFT & DFT experiment
data_path = "database/chat/corpora.json"
import json
from datasets import Dataset
import datasets
from src.dataset.feedback_utils_v2 import Feedback
from src.dataset.format_v2 import to_distill_sft, clean_chat_copora
feedback = Feedback(content = "Role play as a philippine customer, chatting with FWD insurance agent")

dataset = to_distill_sft(feedback)

Loaded 3253 prompts
Loaded 647 search infos


In [3]:
# Self-Distillation FineTuning Model
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# config = PeftConfig.from_pretrained("Ksgk-fy/feedback-adaptor-dft_1")
base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

base_model = AutoModelForCausalLM.from_pretrained(base_model_id)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

if "Meta-Llama-3-" in base_model_id:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    base_model.resize_token_embeddings(len(tokenizer))
else:
    tokenizer.pad_token = tokenizer.unk_token
    
model = PeftModel.from_pretrained(base_model, "Ksgk-fy/feedback-adaptor-dft_1")

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

adaptor_id = "Ksgk-fy/feedback-adaptor-dft"
config = PeftConfig.from_pretrained(adaptor_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, load_in_4bit=True, device_map="auto").eval()
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, adaptor_id, adapter_name="feedback-adaptor-dft")